# Define global hyperparameters:

In [2]:
import numpy as np
import random
import os
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as T
import torch
from torchvision.io import read_image
from PIL import Image
import matplotlib.pyplot as plt

MOMENTUM = 0.9
LEARNING_RATE = 1e-3
BETAS = (0.9, 0.999)
BATCH_SIZE = 64

seed = 6526026
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

# Define custom classes:

In [3]:


class Classifier(nn.Module):
    def __init__(self):
        super(Classifier, self).__init__()
        self.resnet = torch.hub.load("pytorch/vision:v0.13.1", "resnet34", weights='IMAGENET1K_V1')
        self.fc = nn.Linear(1000, 10)
    
    def forward(self, x):
        # NOTE: Flatten has been done
        features = self.resnet(x)
        x = self.fc(features)
        # output = F.log_softmax(x, dim=1)
        return x, features
    
class KMNIST(Dataset):
    def __init__(self, file_path, transforms):
        CHOSEN_CLASSES = {
        "backpack" : 0,
        "book" : 1,
        "car" : 2,
        "pizza" : 3,
        "sandwich" : 4,
        "snake" : 5,
        "sock" : 6,
        "tiger" : 7,
        "tree" : 8,
        "watermelon" : 9,
        }
        labels = []
        imgs = []
        for path, subdirs, files in os.walk(file_path):
            for name in files:
                img_path = os.path.join(path, name)
                c = img_path.split("\\")
                label = CHOSEN_CLASSES[c[1]]
                imgs.append(img_path)
                labels.append(label)
        assert len(labels) == len(imgs)
        self.labels = labels
        self.imgs = imgs
        self.transforms = transforms

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, index):
        img = Image.open(self.imgs[index])
        img = np.array(img)
        label = self.labels[index]
        if self.transforms:
            img = self.transforms(img)
        return img, label

# Construct datasets

In [4]:
train_transforms = T.Compose([
    T.ToTensor(),
    T.ConvertImageDtype(torch.float),
    T.Normalize((0.485, 0.456, 0.406),  (0.229, 0.224, 0.225)),
    T.RandomHorizontalFlip(p=0.5),
    T.Resize(size=(224,224),antialias = False)
])

test_transforms = T.Compose([
    T.ToTensor(),
    T.ConvertImageDtype(torch.float),
    T.Normalize((0.485, 0.456, 0.406),  (0.229, 0.224, 0.225)),
    T.Resize(size=(224,224),antialias = False)
])

real_train = KMNIST(
    "data/real_train",
    train_transforms,
)
real_train_loader = DataLoader(
    dataset = real_train,
    batch_size = BATCH_SIZE,
    shuffle = True,
    num_workers = 0)

real_test = KMNIST(
    "data/real_test",
    test_transforms,
)
real_test_loader = DataLoader(
    dataset = real_test,
    batch_size = BATCH_SIZE,
    shuffle = True,
    num_workers = 0)

real_val = KMNIST(
    "data/real_train",
    test_transforms,
)
real_val_loader = DataLoader(
    dataset = real_val,
    batch_size = BATCH_SIZE,
    shuffle = True,
    num_workers = 0)

sketch_train = KMNIST(
    "data/sketch_train",
    train_transforms,
)
sketch_train_loader = DataLoader(
    dataset = sketch_train,
    batch_size = BATCH_SIZE,
    shuffle = True,
    num_workers = 0)

sketch_test = KMNIST(
    "data/sketch_test",
    test_transforms,
)
sketch_test_loader = DataLoader(
    dataset = sketch_test,
    batch_size = BATCH_SIZE,
    shuffle = True,
    num_workers = 0)

sketch_val = KMNIST(
    "data/sketch_train",
    test_transforms,
)
sketch_val_loader = DataLoader(
    dataset = sketch_val,
    batch_size = BATCH_SIZE,
    shuffle = True,
    num_workers = 0)

# Define training and testing functions:

In [5]:
def test(model, test_loader):
    correct = 0
    total = 0
    criterion = nn.CrossEntropyLoss()
    # since we're not training, we don't need to calculate the gradients for our outputs
    with torch.no_grad():
        for data in test_loader:
            images, labels = data
            images = images.to('cuda') 
            labels = labels.to('cuda') 
            # calculate outputs by running images through the network
            outputs, _ = model(images)
            loss = criterion(outputs, labels)
            # the class with the highest energy is what we choose as prediction
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            
    accuracy = 100 * correct / total 
    return accuracy, loss


def train_model(model, train_loader, val_loader, optimizer, num_epochs=20):
    criterion = nn.CrossEntropyLoss()
    max_acc = 0
    best_epoch = 0
    best_loss = 0
    print("==================Training Model==================")
    for epoch in range(num_epochs):  # loop over the dataset multiple times
        print('------------------Epoch number '+ str(epoch + 1) +'------------------')
        for i, data in enumerate(train_loader):
            # get the inputs; data is a list of [inputs, labels]
            
            inputs, labels = data
            # zero the parameter gradients
            optimizer.zero_grad()

            # forward + backward + optimize
            inputs = inputs.to('cuda') 
            labels = labels.to('cuda') 
            outputs, features = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
        print("\tModel training loss = " + str(loss.item()))
        print("\tEvaluating model...")
        acc, l = test(model, val_loader)
        print("\tEvaluation accuracy " + str(acc)+" , loss = " + str(l.item()))
        if acc > max_acc:
            max_acc = acc
            best_epoch = epoch
            best_loss = l
            torch.save(model.state_dict(), "best_model.pt")
    print('==================Finished Training==================')
    print('\tBest model is epoch number ' + str(best_epoch))
    print('\tBest model evaluation accuracy: ' + str(max_acc))
    print('\tBest model evaluation loss: ' + str(best_loss))
    return model

model = Classifier().cuda()
optimizer = torch.optim.SGD(model.parameters(), lr=LEARNING_RATE, momentum=MOMENTUM)
model = train_model(model, sketch_train_loader, sketch_val_loader, optimizer)
accuracy, loss = test(model, sketch_test_loader)
print("Model trained on sketch_train, testing on sketch_test, accuracy = " + str(accuracy)+" , testing loss = " + str(loss.item()))

Using cache found in C:\Users\Jerry Qian/.cache\torch\hub\pytorch_vision_v0.13.1


==================Training Model==================
------------------Epoch number 1------------------
	Model training loss = 0.6963334679603577
	Evaluating model...
	Evaluation accuracy 86.80981595092024 , loss = 0.4734145700931549
------------------Epoch number 2------------------
	Model training loss = 0.3486490249633789
	Evaluating model...
	Evaluation accuracy 94.68302658486708 , loss = 0.17777997255325317
------------------Epoch number 3------------------
	Model training loss = 0.11765991151332855
	Evaluating model...
	Evaluation accuracy 97.54601226993866 , loss = 0.1550343632698059
------------------Epoch number 4------------------
	Model training loss = 0.11546622216701508
	Evaluating model...
	Evaluation accuracy 98.72188139059304 , loss = 0.04537966102361679
------------------Epoch number 5------------------
	Model training loss = 0.03811189532279968
	Evaluating model...
	Evaluation accuracy 99.5398773006135 , loss = 0.06035955250263214
------------------Epoch number 6-------

In [6]:
model = Classifier().cuda()
optimizer = torch.optim.SGD(model.parameters(), lr=LEARNING_RATE, momentum=MOMENTUM)
model = train_model(model, real_train_loader, sketch_val_loader, optimizer)
accuracy, loss = test(model, sketch_test_loader)
print("Model trained on real_train, testing on sketch_test, accuracy = " + str(accuracy)+" , testing loss = " + str(loss.item()))

Using cache found in C:\Users\Jerry Qian/.cache\torch\hub\pytorch_vision_v0.13.1


==================Training Model==================
------------------Epoch number 1------------------
	Model training loss = 0.19999410212039948
	Evaluating model...
	Evaluation accuracy 56.28834355828221 , loss = 1.6830148696899414
------------------Epoch number 2------------------
	Model training loss = 0.013411101885139942
	Evaluating model...
	Evaluation accuracy 58.640081799591 , loss = 1.7744139432907104
------------------Epoch number 3------------------
	Model training loss = 0.00587375508621335
	Evaluating model...
	Evaluation accuracy 56.95296523517382 , loss = 1.8517279624938965
------------------Epoch number 4------------------
	Model training loss = 0.020555445924401283
	Evaluating model...
	Evaluation accuracy 57.05521472392638 , loss = 2.1892364025115967
------------------Epoch number 5------------------
	Model training loss = 0.0037861159071326256
	Evaluating model...
	Evaluation accuracy 56.441717791411044 , loss = 1.418652057647705
------------------Epoch number 6-----

In [7]:
def train_model_across_domain(model, source_train_loader, target_train_loader, val_loader, optimizer, num_epochs=20):
    criterion = nn.CrossEntropyLoss()
    max_acc = 0
    best_epoch = 0
    best_loss = 0
    print("==================Training Model==================")
    for epoch in range(num_epochs):
        print('------------------Epoch number '+ str(epoch + 1) +'------------------')
        itr = 0
        for source_data, target_data in zip(real_train_loader, sketch_train_loader):
            itr += 1
            if itr >= 30:
                break
            source_inputs, source_labels = source_data
            target_inputs, target_labels = target_data
            source_inputs = source_inputs.to('cuda') 
            source_labels = source_labels.to('cuda') 
            target_inputs = target_inputs.to('cuda') 
            target_labels = target_labels.to('cuda') 
            # zero the parameter gradients
            optimizer.zero_grad()

            # forward + backward + optimize
            outputs, source_features = model(source_inputs)
            loss = criterion(outputs, source_labels)
            _, target_features = model(target_inputs)
            MMD = (source_features/1000 - target_features/1000).pow(2).sum()
            loss = 0.99 * loss + MMD * 0.01
            loss.backward()
            optimizer.step()
        print("\tModel training loss = " + str(loss.item()))
        print("\tEvaluating model...")
        acc, l = test(model, val_loader)
        print("\tEvaluation accuracy " + str(acc)+" , loss = " + str(l.item()))
        if loss < best_loss:
            max_acc = acc
            best_epoch = epoch
            best_loss = l
            torch.save(model.state_dict(), "best_model.pt")
    print('==================Finished Training==================')
    print('\tBest model is epoch number ' + str(best_epoch))
    print('\tBest model evaluation accuracy: ' + str(max_acc))
    print('\tBest model evaluation loss: ' + str(best_loss))
    return model

model = Classifier().cuda()
optimizer = torch.optim.SGD(model.parameters(), lr=LEARNING_RATE, momentum=MOMENTUM)
model = train_model_across_domain(model = model, source_train_loader = real_train_loader, target_train_loader = sketch_train_loader, val_loader = sketch_val_loader, optimizer = optimizer)
accuracy, loss = test(model, sketch_test_loader)
print("Model trained on real_train, testing on sketch_test, accuracy = " + str(accuracy)+" , testing loss = " + str(loss.item()))

Using cache found in C:\Users\Jerry Qian/.cache\torch\hub\pytorch_vision_v0.13.1


==================Training Model==================
------------------Epoch number 1------------------
	Model training loss = 0.11915460228919983
	Evaluating model...
	Evaluation accuracy 53.68098159509202 , loss = 2.1885628700256348
------------------Epoch number 2------------------
	Model training loss = 0.30776286125183105
	Evaluating model...
	Evaluation accuracy 55.11247443762781 , loss = 1.412760615348816
------------------Epoch number 3------------------
	Model training loss = 0.09659098833799362
	Evaluating model...
	Evaluation accuracy 56.441717791411044 , loss = 1.8395886421203613
------------------Epoch number 4------------------
	Model training loss = 0.08528880029916763
	Evaluating model...
	Evaluation accuracy 55.77709611451943 , loss = 2.0007946491241455
------------------Epoch number 5------------------
	Model training loss = 0.032275523990392685
	Evaluating model...
	Evaluation accuracy 57.0040899795501 , loss = 1.4959332942962646
------------------Epoch number 6-------